In [1]:
import pandas as pd
import numpy as np
csv = pd.read_csv('export/15s_133features_filter_std_baseline.csv')
print(f"{csv.shape=}")

def getXY(csv, do_print=True):
    df = csv.copy()
    y_ori = np.array(df['type'])
    y_ori[y_ori == 0] = 0
    y_ori[y_ori == 2] = 1
    y_ori
    head = list(df.keys())
    head.remove('Unnamed: 0')
    head.remove('name')
    head.remove('score')
    head.remove('type')
    X_ori = np.array(df[head])
    if(do_print):
        print(f"{X_ori.shape=} {y_ori.shape=}")
    return X_ori, y_ori

csv.shape=(720, 137)


In [2]:
def train_model(X_ori,y_ori, do_print=True):
    X,y = X_ori.copy(), y_ori.copy()

    from sklearn.svm import SVC
    from sklearn.utils import shuffle
    from sklearn.model_selection import cross_val_score

    X_shuff,y_shuff = shuffle(X,y)
    model = SVC(kernel='rbf')
    cross = cross_val_score(model, X_shuff, y_shuff, cv=10)

    model = SVC(kernel='rbf')
    model.fit(X_shuff, y_shuff)
    ans = model.predict(X_shuff)
    acc = sum(ans == y_shuff) / len(y_shuff)
    if(do_print):
        print(acc, cross.mean(), cross)
        return model
    else:
        return model, acc, cross

def report(names,labels,predicts,do_print = True):
    correct = 0
    wrong = 0
    prev_name = None
    prev_label = None
    count = 0

    table = []
    summary = []

    table.append(f"Name\tLabel\tPredict")
    table.append("="*30)
    for name,predict,label in zip(names,predicts,labels):
        if(predict == label): correct += 1
        else: wrong += 1

        text_predict = 'non-st' if predict == 0 else 'stress'
        text_label = 'non-st' if label == 0 else 'stress'
        if(prev_name != None and prev_name != name and count > 0): 
            text_prev_label = 'non-st' if prev_label == 0 else 'stress'
            summary.append(f"label={text_prev_label} | {count} {prev_name}")
            count = 0
        if(predict != label):
            count = count + 1
            table.append(f"{name}\t{text_label}\t{text_predict}")

        prev_label = label
        prev_name = name
        
    if(do_print):
        print('\n'.join(table))    
        print()
        print("Incorrect summany")
        print("="*30)
        print('\n'.join(summary))
        print(f"Correct={correct} | Wrong={wrong} | Acc={correct / (correct + wrong) * 100}%")
    else:
        acc = correct / (correct + wrong)
        return summary, acc

In [3]:
# model_all
X,y = getXY(csv.copy())
model_all = train_model(X,y)

samples,labels = getXY(csv.copy())
names = csv['name']
predicts = model_all.predict(samples) 
print()
report(names,labels,predicts)

X_ori.shape=(720, 133) y_ori.shape=(720,)
1.0 0.9986111111111111 [1.         0.98611111 1.         1.         1.         1.
 1.         1.         1.         1.        ]
X_ori.shape=(720, 133) y_ori.shape=(720,)

Name	Label	Predict

Incorrect summany

Correct=720 | Wrong=0 | Acc=100.0%


In [7]:
non_stress_names = list(set(csv[csv['type'] == 0]['name']))
non_stress_names.sort()
stress_names = list(set(csv[csv['type'] == 2]['name']))
stress_names.sort()
print(f"{non_stress_names=}\n{stress_names=}")

non_stress_names=['MJ', 'amp', 'beau', 'beer', 'int', 'job', 'miiw', 'mu', 'nisit', 'noey', 'nuclear', 'praewphan', 'prin', 'sam', 'shin', 'suyo']
stress_names=['aui', 'bam', 'bank', 'bas', 'cedric', 'dt', 'eiyu', 'fabby', 'fahmai', 'flm', 'gon', 'kao', 'kee', 'minkhant', 'new', 'pang', 'pla', 'ploy', 'yee', 'yong']


In [8]:
# model_filter
for remove_name in non_stress_names:
    selected_csv = csv.copy()
    cond = selected_csv['name'] != remove_name
    
    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc = report(names,labels,predicts,do_print=False)


    print(f"REMOVE NAME: {remove_name} - ACC = {acc*100}%")
    for text in summary:
        print(f" - {text}")
    # print(" \n".join(summary))
    # print("************************************************************")


REMOVE NAME: MJ - ACC = 98.88888888888889%
 - label=non-st | 8 MJ
REMOVE NAME: amp - ACC = 100.0%
REMOVE NAME: beau - ACC = 100.0%
REMOVE NAME: beer - ACC = 98.61111111111111%
 - label=non-st | 10 beer
REMOVE NAME: int - ACC = 99.16666666666667%
 - label=non-st | 6 int
REMOVE NAME: job - ACC = 97.22222222222221%
 - label=non-st | 20 job
REMOVE NAME: miiw - ACC = 97.22222222222221%
 - label=non-st | 20 miiw
REMOVE NAME: mu - ACC = 97.22222222222221%
 - label=non-st | 20 mu
REMOVE NAME: nisit - ACC = 97.36111111111111%
 - label=non-st | 19 nisit
REMOVE NAME: noey - ACC = 97.22222222222221%
 - label=non-st | 20 noey
REMOVE NAME: nuclear - ACC = 100.0%
REMOVE NAME: praewphan - ACC = 97.77777777777777%
 - label=non-st | 15 praewphan
 - label=non-st | 1 prin
REMOVE NAME: prin - ACC = 97.5%
 - label=non-st | 18 prin
REMOVE NAME: sam - ACC = 99.44444444444444%
 - label=non-st | 4 sam
REMOVE NAME: shin - ACC = 100.0%
REMOVE NAME: suyo - ACC = 98.05555555555556%
 - label=non-st | 14 suyo


In [9]:
# model_filter
for remove_name in stress_names:
    selected_csv = csv.copy()
    cond = selected_csv['name'] != remove_name
    
    X,y = getXY(selected_csv[cond],do_print=False)
    model,acc,cross = train_model(X,y,do_print=False)
    # print(f"ACC = {acc*100}% | CV = {cross.mean()*100}%")

    samples,labels = getXY(csv.copy(),do_print=False)
    names = csv['name']
    predicts = model.predict(samples)
    # print()
    summary,acc = report(names,labels,predicts,do_print=False)


    print(f"REMOVE NAME: {remove_name} - ACC = {acc*100}%")
    for text in summary:
        print(f" - {text}")
    # print(" \n".join(summary))
    # print("************************************************************")


REMOVE NAME: aui - ACC = 99.30555555555556%
 - label=stress | 5 aui
REMOVE NAME: bam - ACC = 97.36111111111111%
 - label=stress | 19 bam
REMOVE NAME: bank - ACC = 97.36111111111111%
 - label=stress | 19 bank
REMOVE NAME: bas - ACC = 100.0%
REMOVE NAME: cedric - ACC = 100.0%
REMOVE NAME: dt - ACC = 97.36111111111111%
 - label=stress | 19 dt
REMOVE NAME: eiyu - ACC = 97.91666666666666%
 - label=stress | 15 eiyu
REMOVE NAME: fabby - ACC = 99.86111111111111%
 - label=stress | 1 fabby
REMOVE NAME: fahmai - ACC = 97.22222222222221%
 - label=stress | 20 fahmai
REMOVE NAME: flm - ACC = 99.16666666666667%
 - label=stress | 6 flm
REMOVE NAME: gon - ACC = 97.77777777777777%
 - label=stress | 16 gon
REMOVE NAME: kao - ACC = 97.91666666666666%
 - label=stress | 15 kao
REMOVE NAME: kee - ACC = 97.22222222222221%
 - label=stress | 20 kee
REMOVE NAME: minkhant - ACC = 99.58333333333333%
 - label=stress | 3 minkhant
REMOVE NAME: new - ACC = 99.16666666666667%
 - label=stress | 6 new
REMOVE NAME: pang -